## Ejercicios de introducción a ProjectQ

1) Escribe una función de nombre hello_entangled_world que reciba un *engine* y devuelva el estado de los qubits después de ejecutar el circuito de la figura.

<img src="bell.png" width=50%>

In [1]:
from projectq.ops import Measure, H, CNOT, All

def hello_entangled_world(eng):
    q = eng.allocate_qureg(2)
    
    H | q[0]
    
    CNOT | (q[0], q[1])
    
    return (q)

2) Usando el simulador, ejecuta el circuito 1000 veces y calcula la frecuencia de cada una de las cuatro posibles salidas. 

In [2]:
import projectq

results = {0:0, 1:0, 10:0, 11:0}

eng = projectq.MainEngine()

for i in range(0, 1000):
    q = hello_entangled_world(eng)
    
    All(Measure) | q
    
    results[10*int(q[0]) + int(q[1])] += 1
    
for key in results:
    print(f"{key}:{results[key]}")


0:484
1:0
10:0
11:516


3) Repite el cálculo de frecuencias, pero esta vez en el ordenador de la IBM Quantum Experience 

In [3]:
import projectq.setups.ibm
from projectq.backends import IBMBackend

eng = projectq.MainEngine(IBMBackend(use_hardware=True, num_runs=1024,verbose=True,
                                     device='ibmqx4', num_retries=30),
                          engine_list=projectq.setups.ibm.get_engine_list())

hello_entangled_world(eng)

eng.flush()

- Authenticating...
IBM QE user (e-mail) > messcoderewe@gmail.com
IBM QE password > ········
- Running code: 
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[1];
cx q[1], q[0];
- There was an error running your code:
400 Client Error: Bad Request for url: https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=JbXVRMhkDGzhFBe6lYReJhBucbRpKFVgHeHIN2JBSdi5cYrP8MtYyXQ8TtkTsjUj&deviceRunType=ibmqx4&fromCache=false&shots=1024


Exception: Failed to run the circuit. Aborting.
 raised in:
'  File "C:\\Users\\pytho\\AppData\\Roaming\\Python\\Python37\\site-packages\\projectq\\backends\\_ibm\\_ibm.py", line 309, in _run'
'    raise Exception("Failed to run the circuit. Aborting.")'

4) Accediendo directamente a la función de onda, calcula las amplitudes y las probabilidades de cada uno de los estados de la base computacional.

In [4]:
eng = projectq.MainEngine()

q = hello_entangled_world(eng)

eng.flush()

def printWave(eng, val, q):
    print(f"{val} amplitude: {eng.backend.get_amplitude(val, q)}, probability: {eng.backend.get_probability(val, q)}")
    
for val in ['00','01','10','11']:
    printWave(eng, val, q)
        

00 amplitude: (0.7071067811865475+0j), probability: 0.4999999999999999
01 amplitude: 0j, probability: 0.0
10 amplitude: 0j, probability: 0.0
11 amplitude: (0.7071067811865475+0j), probability: 0.4999999999999999


5) ¿Crees que podrías diseñar un circuito que entrelazara 3 qubits de forma que su estado fuera $\frac{1}{\sqrt{2}}(|000\rangle+|111\rangle)$? 

In [6]:
def entangle_3(eng):
    q = eng.allocate_qureg(3)
    
    H | q[0]
    
    CNOT | (q[0], q[1])
    CNOT | (q[1], q[2])
    
    return q

eng = projectq.MainEngine()

q = entangle_3(eng)
eng.flush()

for val in ['000', '001', '010', '011', '100', '101', '110', '111']:
    printWave(eng, val, q)
    
All(Measure) | q


000 amplitude: (0.7071067811865475+0j), probability: 0.4999999999999999
001 amplitude: 0j, probability: 0.0
010 amplitude: 0j, probability: 0.0
011 amplitude: 0j, probability: 0.0
100 amplitude: 0j, probability: 0.0
101 amplitude: 0j, probability: 0.0
110 amplitude: 0j, probability: 0.0
111 amplitude: (0.7071067811865475+0j), probability: 0.4999999999999999


6) Comprueba la siguiente equivalencia de puertas $$ Z = H X H$$

Para ello, ten en cuenta que es necesario y suficiente comprobar que dan el mismo resultado (obtienen el mismo estado, no sólo las mismas probabilidades) para los valores iniciales del qubit $|0\rangle$ y $|1\rangle$.

In [7]:
from projectq.ops import Z, H, X, All
from itertools import product

def apply_z(eng, q):
    Z | q
    
def apply_hxh(eng, q):
    H | q
    X | q
    H | q
    
def test(func, numqubit = 1):
    
    print("Testing function: " + func.__name__)
    
    eng = projectq.MainEngine()
    q = eng.allocate_qureg(numqubit)
    
    func(eng, q)
    eng.flush()
    
    bin_values = ['0', '1']
    for val_list in product(bin_values, repeat=numqubit):
        val = ''.join(val_list)
        printWave(eng, val, q)
        
    All(Measure) | q
    
for f in [apply_z, apply_hxh]:
    
    test(f)


Testing function: apply_z
0 amplitude: (1+0j), probability: 1.0
1 amplitude: 0j, probability: 0.0
Testing function: apply_hxh
0 amplitude: (0.9999999999999998+0j), probability: 0.9999999999999996
1 amplitude: 0j, probability: 0.0


7) Comprueba también la siguiente equivalencia de puertas $$S^\dagger = S S S$$

In [8]:
from projectq.ops import S, DaggeredGate

def inverse_S(eng, q):
    DaggeredGate(S) | q
    
def triple_S(eng, q):
    S | q
    S | q
    S | q
    
for f in [inverse_S, triple_S]:
    test(f)
    

Testing function: inverse_S
0 amplitude: (1+0j), probability: 1.0
1 amplitude: 0j, probability: 0.0
Testing function: triple_S
0 amplitude: (1+0j), probability: 1.0
1 amplitude: 0j, probability: 0.0


8) ¿Qué función calcula el circuito de la figura? Ten en cuenta que ahora hay cuatro posibles valores iniciales para los qubits: $|00\rangle,|01\rangle,|10\rangle$ y $|11\rangle$.

<img src="circuito8.png" width=50%>

In [9]:
# q[0] se mide al inicio, si resulta en 1 medirá q[1] y aplicará un NOT sobre él.
# si q[0] mide 0, q[1] será medido a continuación y de medir 1 aplicará un NOT sobre q[0].
#
# Inicial: |00> Se estabilizarán como tal.
# Inicial: |01> Se aplicará un NOT |10>
# Inicial: |10> Se aplicará un NOT |01>
# Inicial: |11> Se estabilizarán como tal.
#
# Esta puerta reordena los qubits, es la puerta Swap

from projectq.ops import CNOT, X

def func_misteriosa(eng, q):
    CNOT | (q[0], q[1])
    CNOT | (q[1], q[0])
    CNOT | (q[0], q[1])
    
def fm_00(eng, q):
    func_misteriosa(eng, q)
    
def fm_01(eng, q):
    X | q[1]
    func_misteriosa(eng, q)
    
def fm_10(eng, q):
    X | q[0]
    func_misteriosa(eng, q)
    
def fm_11(eng, q):
    X | q[0]
    X | q[1]
    func_misteriosa(eng, q)
    
for f in [fm_00, fm_01, fm_10, fm_11]:
    test(f, 2)
    

Testing function: fm_00
00 amplitude: (1+0j), probability: 1.0
01 amplitude: 0j, probability: 0.0
10 amplitude: 0j, probability: 0.0
11 amplitude: 0j, probability: 0.0
Testing function: fm_01
00 amplitude: 0j, probability: 0.0
01 amplitude: 0j, probability: 0.0
10 amplitude: (1+0j), probability: 1.0
11 amplitude: 0j, probability: 0.0
Testing function: fm_10
00 amplitude: 0j, probability: 0.0
01 amplitude: (1+0j), probability: 1.0
10 amplitude: 0j, probability: 0.0
11 amplitude: 0j, probability: 0.0
Testing function: fm_11
00 amplitude: 0j, probability: 0.0
01 amplitude: 0j, probability: 0.0
10 amplitude: 0j, probability: 0.0
11 amplitude: (1+0j), probability: 1.0


9) Determina qué función es calculada por el circuito de la figura. De nuevo, hay cuatro posibles valores iniciales para los qubits.

<img src="circuito9.png" width=50%>

In [10]:
# Primero se causa una superposición independiente en ambos qubits.
# Después se mide q[1], invirtiendo q[0] si es 1.
# A continuación se deshace la superposición inicial.
#
# El resultado es simplemente negar el primer qubit en función del 
# segundo.

from projectq.ops import CNOT, H

def circuito(eng, q):
    H | q[0]
    H | q[1]
    CNOT | (q[1], q[0])
    H | q[0]
    H | q[1]
    
def c_00(eng, q):
    circuito(eng, q)
    
def c_01(eng, q):
    X | q[1]
    circuito(eng, q)
    
def c_10(eng, q):
    X | q[0]
    circuito(eng, q)
    
def c_11(eng, q):
    X | q[0]
    X | q[1]
    circuito(eng, q)
    
for f in [c_00, c_01, c_10, c_11]:
    test(f, 2)

Testing function: c_00
00 amplitude: (0.9999999999999997+0j), probability: 0.9999999999999993
01 amplitude: 0j, probability: 0.0
10 amplitude: 0j, probability: 0.0
11 amplitude: 0j, probability: 0.0
Testing function: c_01
00 amplitude: 0j, probability: 0.0
01 amplitude: (0.9999999999999997+0j), probability: 0.9999999999999993
10 amplitude: 0j, probability: 0.0
11 amplitude: 0j, probability: 0.0
Testing function: c_10
00 amplitude: 0j, probability: 0.0
01 amplitude: 0j, probability: 0.0
10 amplitude: 0j, probability: 0.0
11 amplitude: (0.9999999999999997+0j), probability: 0.9999999999999993
Testing function: c_11
00 amplitude: 0j, probability: 0.0
01 amplitude: 0j, probability: 0.0
10 amplitude: (0.9999999999999997+0j), probability: 0.9999999999999993
11 amplitude: 0j, probability: 0.0


10) Comprueba que el circuito de la figura es equivalente a una puerta de Toffoli (ahora el número de posibles valores inciales que debes comprobar es 8, porque hay tres qubits). 

<img src="toffoli.png" width=80%>

In [11]:
# El circuito mostrado es aproximadamente equivalente a una
# puerta Toffoli, aunque introduce una probabilidad de error
# del orden 10 ^ -32

from projectq.ops import H, CNOT, T, get_inverse, Toffoli

def test_params(functions, numqubit):
    
    bin_values = ['0', '1']
    for val_list in product(bin_values, repeat=numqubit):
        
        print(f"Params: {val_list}")
        
        for func in functions:
            
            for i, val in enumerate(val_list):
                if (i == '1'):
                    X | q[i]

            test(func, numqubit)

def alt_toffoli(eng, q):
    Tdg = get_inverse(T)
    
    H | q[0]
    CNOT | (q[1], q[0])
    Tdg | q[0]
    CNOT | (q[2], q[0])
    T | q[0]
    CNOT | (q[1], q[0])
    Tdg | q[0]
    CNOT | (q[2], q[0])
    T | q[0]
    T | q[1]
    H | q[0]
    CNOT | (q[2], q[1])
    Tdg | q[1]
    T | q[2]
    CNOT | (q[2], q[1])
    
def toffoli(eng, q):
    Toffoli | ((q[2], q[1]), q[0])
    
test_params([alt_toffoli, toffoli], 3)


Params: ('0', '0', '0')
Testing function: alt_toffoli
000 amplitude: (1+0j), probability: 1.0
001 amplitude: 0j, probability: 0.0
010 amplitude: 0j, probability: 0.0
011 amplitude: 0j, probability: 0.0
100 amplitude: (-2.220446049250313e-16+0j), probability: 4.930380657631324e-32
101 amplitude: 0j, probability: 0.0
110 amplitude: 0j, probability: 0.0
111 amplitude: 0j, probability: 0.0
Testing function: toffoli
000 amplitude: (1+0j), probability: 1.0
001 amplitude: 0j, probability: 0.0
010 amplitude: 0j, probability: 0.0
011 amplitude: 0j, probability: 0.0
100 amplitude: 0j, probability: 0.0
101 amplitude: 0j, probability: 0.0
110 amplitude: 0j, probability: 0.0
111 amplitude: 0j, probability: 0.0
Params: ('0', '0', '1')
Testing function: alt_toffoli
000 amplitude: (1+0j), probability: 1.0
001 amplitude: 0j, probability: 0.0
010 amplitude: 0j, probability: 0.0
011 amplitude: 0j, probability: 0.0
100 amplitude: (-2.220446049250313e-16+0j), probability: 4.930380657631324e-32
101 amplitud

11) Crea un circuito con una sola puerta de Toffoli. Si lo lanzas a ejecutar en IBM Quantum Experience, ¿en qué puertas se transforma?

In [12]:
# Se transforma en otra serie de puertas equivalentes
# distintas a las mostradas en el diagrama superior.
# 
# qreg q[3];
# creg c[3];
#
# h q[0];
# cx q[1], q[0];
# tdg q[0];
# cx q[2], q[0];
# t q[0];
#
# Aquí comienzan las diferencias:
# t q[1];
# cx q[2], q[1];
# tdg q[1];
# cx q[2], q[1];
# cx q[1], q[0];
# tdg q[0];
# cx q[2], q[0];
# t q[0];
# h q[0];
# t q[2];

eng = projectq.MainEngine(IBMBackend(use_hardware=True, num_runs=1024,verbose=True,
                                     device='ibmqx4', num_retries=30),
                          engine_list=projectq.setups.ibm.get_engine_list())

q = eng.allocate_qureg(3)
toffoli(eng, q)

eng.flush()

- Authenticating...
IBM QE user (e-mail) > messcoderewe@gmail.com
IBM QE password > ········
- Running code: 
include "qelib1.inc";
qreg q[3];
creg c[3];
h q[0];
cx q[1], q[0];
tdg q[0];
cx q[2], q[0];
t q[0];
t q[1];
cx q[2], q[1];
tdg q[1];
cx q[2], q[1];
cx q[1], q[0];
tdg q[0];
cx q[2], q[0];
t q[0];
h q[0];
t q[2];
- There was an error running your code:
400 Client Error: Bad Request for url: https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=ImGqnGbiej6IF3LzIrOY9GbPYTWSruDuSOtb6BdINmKYqQgyukYmxCBFZh3PUYaP&deviceRunType=ibmqx4&fromCache=false&shots=1024


Exception: Failed to run the circuit. Aborting.
 raised in:
'  File "C:\\Users\\pytho\\AppData\\Roaming\\Python\\Python37\\site-packages\\projectq\\backends\\_ibm\\_ibm.py", line 309, in _run'
'    raise Exception("Failed to run the circuit. Aborting.")'

12) Comprobemos los límites del simulador de ProjectQ. Crea una función que reciba un parámetro $n$ entero y cree un circuito en el que se aplica la puerta $H$ a cada uno de los qubits y luego se aplica la puerta $CNOT$ a cada par de qubits consecutivos. Finalmente, se miden todos los qubits. ¿Cuál es el mayor $n$ para el que se puede ejecutar el circuito en el simulador?

In [ ]:
def stress(n):
    print(f"Intentando ejecutar prueba con n={n}")
    
    eng = projectq.MainEngine()
    q = eng.allocate_qureg(n)
    
    for i in range(n):
        H | q[i]
        
    for i in range(int(n/2)):
        CNOT | (q[i*2 + 1], q[i*2])
        
    All(Measure) | q
    eng.flush()
    
    print("Ejecución terminada")

In [18]:
# Mi ordenador se congela al intentar ejecutar n=100.
#
# Tras unos 5 minutos de procesamiento la memoria se llena
# y la ejecución falla.

'''
for i in range(30):
    print(f"--- 10 ^ {i}")
    stress(int(10 ** i))

'''    

--- 10 ^ 0
Intentando ejecutar prueba con n=1
Ejecución terminada
--- 10 ^ 1
Intentando ejecutar prueba con n=10
Ejecución terminada
--- 10 ^ 2
Intentando ejecutar prueba con n=100


MemoryError: bad allocation
 raised in:
'  File "C:\\Users\\pytho\\AppData\\Roaming\\Python\\Python37\\site-packages\\projectq\\backends\\_sim\\_simulator.py", line 382, in _handle'
'    self._simulator.allocate_qubit(ID)'